In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import h5py
import numpy as np

sys.path.append("..")
import axionmirror.units_constants as uc

from config import pc_dict

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps as cms
mpl.rc_file('../matplotlibrc')

In [62]:
nu_ref = 400. # [MHz]
t_ref = pc_dict['CHIME'].telescope
sens_ref = t_ref.sens_estimate([nu_ref])[0]
merit_ref = t_ref.merit(nu_ref)

print(f'At {nu_ref:.0f} MHz')
print('name     Aeff[m^2]  FOV[deg^2] n_pixel tobs/day[min] sqrt(npol*Aeff*FOV) sens(rel.)')
pc_names = ['CHIME', 'HERA', 'CHORD', 'HIRAX256', 'HIRAX1024', 'BURSTT256', 'BURSTT2048']

for pc_name in pc_names:
    pc = pc_dict[pc_name]
    t = pc.telescope
    
    print(f'{t.name:12}', end='')
    A = t.Aeff_zenith(nu_ref)/10000 # [m^2]
    print(f'{A:6.0f}', end='')
    FOV = t.instantaneous_fov(nu_ref)*(180/np.pi)**2 # [deg^2]
    print(f'{FOV:12.0f}', end='')
    pixel_size = (uc.c0 / nu_ref) ** 2 / t.Aeff_zenith(nu_ref) # [rad^2]
    n_pixel = t.survey_area(nu_ref) / pixel_size
    print(f'{n_pixel:8.0f}', end='')
    print(f'{t.t_obs(nu_ref, t.dec)/t.t_obs_days/60:14.1f}', end='')
    print(f'{t.merit(nu_ref) / merit_ref:15.3f}', end='')
    print(f'{(t.sens_estimate([nu_ref])[0] / sens_ref) ** (-2):11.3f}')

At 400 MHz
name     Aeff[m^2]  FOV[deg^2] n_pixel tobs/day[min] sqrt(npol*Aeff*FOV) sens(rel.)
CHIME         3120         515   41379          26.3          1.000      1.000
HERA         29548          16   19637          18.4          0.537      0.323
CHORD         7173         102   52317          10.5          0.676      0.815
HIRAX256      3629         102   34902           7.9          0.481      0.446
HIRAX1024    14515         102  139607           7.9          0.962      0.892
BURSTT256      131        3600    1343         262.1          0.383      0.462
BURSTT2048    1037        3600   10624         262.1          1.078      1.299


In [4]:
print('name     Central freq. Aeff[m^2]  FOV[deg^2] sqrt(npol*Aeff*FOV) sens(rel.)')
pc_names = ['CHIME', 'HERA', 'HERA', 'CHORD', 'HIRAX256', 'HIRAX1024', 'BURSTT256', 'BURSTT2048']

HERA_counter = 0
for pc_name in pc_names:

    pc = pc_dict[pc_name]

    t = pc.telescope
    print(f'{t.name:12}', end='')

    nu = 400.
    if pc_name == 'HERA':
        HERA_counter += 1
        nu = 250. if HERA_counter == 1 else 400.
    print(f'{nu:8.0f}', end='')

    A = t.Aeff_zenith(nu)/10000 # [m^2]
    print(f'{A:6.0f}', end='')

    FOV = t.instantaneous_fov(nu)*(180/np.pi)**2 # [deg^2]
    print(f'{FOV:12.0f}', end='')
    
    merit = t.merit(nu)
    if pc_name == 'CHIME':
        merit_ref = merit
    print(f'{merit / merit_ref:15.3f}', end='')

    with h5py.File(f"../outputs/plot_data/{pc.name}/total-base.h5", 'r') as hf:
        nu_arr = hf['nu'][:]
        gagg_arr = hf['gagg'][:]

    gagg_med = np.median(gagg_arr, axis=0)
    sens_est = np.interp(nu, nu_arr, gagg_med)
    if pc_name == 'CHIME':
        sens_est_ref = sens_est
    print(f'{(sens_est / sens_est_ref)**(-2):11.3f}')

name     Central freq. Aeff[m^2]  FOV[deg^2] sqrt(npol*Aeff*FOV) sens(rel.)
CHIME            400  3120         515          1.000      1.000
HERA             250 29548          40          0.859      1.374
HERA             400 29548          16          0.537      1.374
CHORD            400  7173         102          0.676      0.993
HIRAX256         400  3629         102          0.481      0.684
HIRAX1024        400 14515         102          0.962      1.834
BURSTT256        400   131        3600          0.383      0.251
BURSTT2048       400  1037        3600          1.078      1.258


In [15]:
1.374 * (400**-3/(35*(400/408)**-2.5+100)) / (250**-3/(35*(250/408)**-2.5+100))

0.5373220202804903

In [10]:
with h5py.File(f"../outputs/plot_data/{pc_dict['HIRAX1024'].name}/total-base.h5", 'r') as hf:
    nu_arr = hf['nu'][:]
    gagg_arr = hf['gagg'][:]

print(np.median(gagg_arr[:,0]))

1.2816486840672586e-11


In [11]:
with h5py.File(f"../outputs/plot_data/{pc_dict['CHIME'].name}/total-base.h5", 'r') as hf:
    nu_arr = hf['nu'][:]
    gagg_arr = hf['gagg'][:]

print(np.median(gagg_arr[:,0]))

1.7354508929977476e-11


In [12]:
with h5py.File(f"../outputs/plot_data/{pc_dict['HIRAX256'].name}/total-base.h5", 'r') as hf:
    nu_arr = hf['nu'][:]
    gagg_arr = hf['gagg'][:]
    
print(np.median(gagg_arr[:,0]))

2.0978565990586333e-11


In [13]:
(2.0978565990586333e-11/1.2816486840672586e-11)**2

2.679252253353275